In [4]:
# A bunch of imports
from stackapi import StackAPI
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [8]:
SITE = StackAPI('stackoverflow')

In [3]:
# set SO parameters
SITE.page_size= 100
SITE.max_pages = 2
post = SITE.fetch('questions',fromdate=datetime(2019,1,1), todate=datetime(2019,12,31))

In [4]:
data = post['items'] # extract only the meaningful info

In [6]:
print(data[0])

{'tags': ['python', 'python-3.x', 'pyhook'], 'owner': {'reputation': 25, 'user_id': 10517777, 'user_type': 'registered', 'profile_image': 'https://www.gravatar.com/avatar/f3c8209b246c8fd97fa5e2263e4c48e2?s=128&d=identicon&r=PG&f=1', 'display_name': 'sergioMoreno', 'link': 'https://stackoverflow.com/users/10517777/sergiomoreno'}, 'is_answered': False, 'view_count': 75, 'answer_count': 0, 'score': 2, 'last_activity_date': 1582123125, 'creation_date': 1576504384, 'last_edit_date': 1582123125, 'question_id': 59358048, 'link': 'https://stackoverflow.com/questions/59358048/pyhook-crashes-when-it-tries-to-interpret-the-window-name-as-ascii', 'title': 'Pyhook crashes when it tries to interpret the window name as ASCII'}


In [12]:
# write all useful info to a csv file
post_dict = { 'owner_id': [],'owner_url':[],'owner_type':[],'owner_repute':[],
             'tags':[], 'score':[],'view_count':[],'answer_count':[], 
             'last_edit_date':[],'last_activity_date':[],'creation_date':[],
             'q_id':[],'q_link':[],'q_title':[]}

for e in data:
    
    # adding the tag data
    cur_tag = ''
    for t in range(0,len(e['tags'])):
        cur_tag+=e['tags'][t]
        if t < len(e['tags'])-1:
            cur_tag+="&"
    post_dict['tags'].append(cur_tag)
    
    # add owner details
    if 'user_id' in e['owner']:
        post_dict['owner_id'].append(e['owner']['user_id'])
    else:
        post_dict['owner_id'].append('NO-DATA')
    
    if 'link' in e['owner']:
        post_dict['owner_url'].append(e['owner']['link'])
    else:
        post_dict['owner_url'].append('NO-DATA')
    
    if 'user_type' in e['owner']:
        post_dict['owner_type'].append(e['owner']['user_type'])
    else:
        post_dict['owner_type'].append('NO-DATA')
    
    if 'reputation' in e['owner']:
        post_dict['owner_repute'].append(e['owner']['reputation'])
    else:
        post_dict['owner_repute'].append('NO-DATA')
    
    #post level stats
    post_dict['score'].append(e['score'])
    post_dict['view_count'].append(e['view_count'])
    post_dict['answer_count'].append(e['answer_count'])
    
    # dates associated with the posts
    post_dict['creation_date'].append(e['creation_date'])
    if 'last_edit_date' in e.keys():
        post_dict['last_edit_date'].append(e['last_edit_date'])
    else:
        post_dict['last_edit_date'].append('NO-DATA')
    
    if 'last_activity_date' in e.keys():
        post_dict['last_activity_date'].append(e['last_activity_date'])
    else:
        post_dict['last_activity_date'].append('NO-DATA')
                       
    #post identifiers
    post_dict['q_id'].append(e['question_id'])
    post_dict['q_link'].append(e['link'])
    post_dict['q_title'].append(e['title'])
                                     

In [14]:
post_frame = pd.DataFrame(post_dict)

In [51]:
post_frame.to_csv("Post_stats_2.19.2020.csv")

NameError: name 'post_frame' is not defined

In [1]:
## create a list of comments, answers from the questions

In [52]:
data = pd.read_csv("SO_SMSposter_data.csv")

In [53]:
cur_ids = data['q_id'].to_list() # get all the question ids

In [54]:
print(cur_ids)

[58747823, 57277759, 54365906, 58386069, 56920061, 58811819, 58387606, 54757386, 55843097, 56578783, 54409874, 58127896, 56166319, 55496289, 58321114]


In [55]:
# get all answers from these questions - This method has a limit of 100 calls
answers = SITE.fetch('questions/{ids}/answers', ids = cur_ids)
#get all the comments associated with the questions
comm_ques = SITE.fetch('questions/{ids}/comments', ids = cur_ids)

In [57]:
ans_data = answers['items']

In [59]:
ans_ids = [ans_data[k]['answer_id'] for k in range(0,len(ans_data))] 

In [92]:
ans_ques = {}
for e in ans_data:
    cur_aid = e['answer_id']
    cur_qid = e['question_id']
    ans_ques[cur_aid] = cur_qid

In [93]:
ans_ques

{60322295: 58747823,
 60318566: 55496289,
 60295738: 58747823,
 60302130: 58811819,
 60300583: 58127896,
 60300824: 55496289,
 60300784: 54757386,
 60300617: 55843097,
 60273577: 58387606,
 60169430: 58321114,
 57285217: 57277759,
 56579842: 56578783,
 60299160: 54409874,
 60299012: 54365906,
 59176682: 56920061,
 60275193: 58387606,
 60274869: 58747823,
 60263244: 58321114,
 60222968: 58321114,
 60182305: 56166319,
 60162845: 58321114,
 60141698: 56166319,
 59345120: 58321114,
 59964209: 55496289,
 59959115: 55496289,
 59916899: 55496289,
 59913261: 58321114,
 59890057: 58127896,
 59814352: 58321114,
 59617351: 58321114,
 59506642: 54409874,
 58539626: 58321114,
 59303185: 58387606,
 59299937: 56166319,
 59278113: 58321114,
 59261764: 56166319,
 55496390: 55496289,
 58409339: 58127896,
 59023973: 58127896,
 58986880: 55496289,
 58948540: 55496289,
 58569057: 55843097,
 58888227: 55496289,
 58774271: 58321114,
 58812069: 58811819,
 58828202: 58127896,
 58814466: 58811819,
 58811894: 58

In [77]:
# get all the ids of the comments associated with the answers
comm_ans = SITE.fetch('answers/{ids}/comments', ids = ans_ids)
                                     

In [81]:
commans_data = comm_ans['items']
commques_data = comm_ques['items']

In [82]:
commques_data[0]

{'owner': {'reputation': 503,
  'user_id': 3350692,
  'user_type': 'registered',
  'profile_image': 'https://www.gravatar.com/avatar/f4a9c5ca20b7d856efc38bafb3c78626?s=128&d=identicon&r=PG',
  'display_name': 'tudor.gergely',
  'link': 'https://stackoverflow.com/users/3350692/tudor-gergely'},
 'edited': False,
 'score': 0,
 'creation_date': 1582182716,
 'post_id': 58747823,
 'comment_id': 106690990}

In [96]:
# aggregate all the answers at the question level
ques_ans = {}
for e in ans_ques:
    qid = ans_ques[e]
    if qid not in ques_ans:
        ques_ans[qid] = []
        ques_ans[qid].append(e)
    else:
        ques_ans[qid].append(e)
#ques_ans

In [98]:
# combine all comments under question, answers
ques_comm = {}
comm_ans = {} # only for those that given to answers

for e in commques_data:
    cid = e['comment_id']
    qid = e['post_id']
    if qid not in ques_comm:
        ques_comm[qid] = []
        ques_comm[qid].append(cid)
    else:
        ques_comm[qid].append(cid)
    
for e in commans_data:
    cid = e['comment_id']
    aid = e['post_id']
    qid = ans_ques[aid]
    if qid not in ques_comm:
        ques_comm[qid] = []
        ques_comm[qid].append(cid)
    else:
        ques_comm[qid].append(cid)
    if cid not in comm_ans:
        comm_ans[qid] = []
        comm_ans[qid].append(aid)
    else:
        comm_ans[qid].append(aid)

In [109]:
len(ques_comm)

14

In [104]:
urls = data['q_links'].to_list()

KeyError: 'q_links'

In [112]:
data_dict = {'ques_id':[],'url':[],'answers':[],'comments':[]}
urls = data['q_link']
for e in range(0,len(cur_ids)):
    ques_id = cur_ids[e]
    data_dict['ques_id'].append(ques_id)
    data_dict['url'].append(urls[e])
    
    ans = ''
    for a in range(0,len(ques_ans[ques_id])):
        ans+=str(ques_ans[ques_id][a])
        if a<len(ques_ans[ques_id])-1:
            ans+="&"
    #print(ans)
    data_dict['answers'].append(ans)
    
    comms = ''
    if ques_id in ques_comm:
        comments = ques_comm[ques_id]
        #print(comments)
        for a in range(0,len(ques_comm[ques_id])):
            comms+=str(ques_comm[ques_id][a])
            if a<len(ques_comm[ques_id])-1:
                comms+="&"
        data_dict['comments'].append(comms)
    else:
        data_dict['comments'].append('NO_DATA')

post_data = pd.DataFrame(data_dict)

In [113]:
post_data.to_csv("SO_smsposter_postaggdata.csv")